In [ ]:
import cv2
import numpy as np    #import

In [ ]:
img = cv2.imread('panther.jpg', cv2.IMREAD_GRAYSCALE)
img = cv2.resize(img, (800,615), interpolation = cv2.INTER_AREA)    #load image

#cv2.imshow('original image', img)
#cv2.waitKey(0)
#cv2.destroyAllWindows()

In [ ]:
noise = np.zeros(img.shape, np.int16)
cv2.randn(noise, 0.0, 125.0)
gauss = cv2.add(img, noise, dtype=cv2.CV_8UC1)    #normal distribution

In [ ]:
#salt and pepper noise
def addPointNoise(img, percentage, value):
    noise = np.copy(img)
    n = int(img.shape[0] * img.shape[1] * percentage)    #number of 'broken detector pixels'

    for k in range(1, n):
        i = np.random.randint(0, img.shape[1])
        j = np.random.randint(0, img.shape[0])    #choosing pixel

        if img.ndim == 2:
            noise[j, i] = value

        if img.ndim == 3:
            noise[j, i] = [value, value, value]

    return noise

def addSaltAndPepperNoise(img, percentage1, percentage2):
    n = addPointNoise(img, percentage1, 255)
    n2 = addPointNoise(n, percentage2, 0)
    return n2

In [ ]:
#poisson noise
def sp_noise(img, prob):
    noise = np.random.poisson(prob, img.shape)
    output = img + noise
    output = 255 * (output - output.min()) / (output.max() - output.min())
    output = np.array(output, dtype='u1')    #dtype important

    return output

In [ ]:
#shot, salt and pepper
sandp = addSaltAndPepperNoise(img, 0.05, 0.05)

shot = sp_noise(img, 1000)

In [ ]:
cv2.imshow('original', img)
cv2.imshow('poisson noise', shot)
cv2.imshow('salt and pepper noise', sandp)    #print images
cv2.imshow('gauss noise', gauss)

cv2.waitKey(0)     #press '0' to close all
cv2.destroyAllWindows()

#cv2.imwrite('original.png',img)
#cv2.imwrite('poisson.png',shot)
#cv2.imwrite('salt_and_p.png',sandp)    #save
#cv2.imwrite('gauss.png',gauss)

In [ ]:
#what to do with the noise

In [ ]:
#import cv2
#import numpy as np    #import

#img = cv2.imread('panther.jpg', cv2.IMREAD_GRAYSCALE)
#img = cv2.resize(img, (800,615), interpolation = cv2.INTER_AREA)    #load image

#noise = np.zeros(img.shape, np.int16)
#cv2.randn(noise, 0.0, 100.0)
#gauss = cv2.add(img, noise, dtype=cv2.CV_8UC1)    #normal distribution


sharpen_kernel = np.array([0, -1, 0, -1, 5, -1, 0, -1, 0]).reshape((3, 3, 1))    #define matrix
sharpen_kernel = sharpen_kernel
sharpen_kernel_2 = sharpen_kernel*1.5

convolution = cv2.blur(gauss, (9, 9))
gaussconvolution = cv2.GaussianBlur(gauss, (9, 9), sigmaX=2.0, sigmaY=2.0)    #convolution
sharpened_img = cv2.filter2D(gaussconvolution, -1, sharpen_kernel)
sharpened_img_2 = cv2.filter2D(gaussconvolution, -1, sharpen_kernel_2)


cv2.imshow('noisy image', gauss)
#cv2.imshow('blur convolution', convolution)
cv2.imshow('gauss convolution', gaussconvolution)    #print
#cv2.imshow('gauss sharpened', sharpened_img)
cv2.imshow('gauss sharpened_2', sharpened_img_2)

cv2.waitKey(0)
cv2.destroyAllWindows()

#cv2.imwrite('noisy_image.png',gauss)
#cv2.imwrite('gauss_sharpened.png',sharpened_img_2)    #save

In [ ]:
#edge detection

In [ ]:
MAGN_THRESH_PERCENT = 0.2

#img = cv2.imread('panther.jpg', cv2.IMREAD_GRAYSCALE)    #load image
#img = cv2.resize(img, (800,615), interpolation = cv2.INTER_AREA)

def display_image(window, image):
    disp = cv2.normalize(image, None, 0, 255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8UC1)    #normalize and show image
    cv2.imshow(window, disp)


ksize = 3 #convolution size

Ix = cv2.Sobel(img, cv2.CV_32FC1, 1, 0, None, ksize)    #approximation of the partial derivate in the x direction
display_image('Ix', Ix)

Iy = cv2.Sobel(img, cv2.CV_32FC1, 0, 1, None, ksize)    #y direction
display_image('Iy', Iy)

Imagn = cv2.magnitude(Ix, Iy)
display_image('Gradient magnitude', Imagn)    #detected edges

magn_th = np.amax(Imagn) * MAGN_THRESH_PERCENT
print('magn_th =', magn_th)
_, ImagnTh = cv2.threshold(Imagn, magn_th, 1.0, cv2.THRESH_BINARY)    #treshold for true edges
ImagnTh = np.array(ImagnTh, dtype='u1')    #dtype for image saving
display_image('Thresholded gradient magnitude', ImagnTh)  

cv2.waitKey(0)
cv2.destroyAllWindows()

#cv2.imwrite('Ix.png',Ix)
#cv2.imwrite('Iy.png',Iy)
#cv2.imwrite('magnitude.png',Imagn)

In [ ]:
#img = cv2.imread('panther.jpg', cv2.IMREAD_GRAYSCALE)    #load image
#img = cv2.resize(img, (800,615), interpolation = cv2.INTER_AREA)

blurred = cv2.GaussianBlur(img, (7, 7), 1.5)
LoG = cv2.Laplacian(blurred, cv2.CV_16S, ksize=3)    #one mask (LoG)

minLoG = cv2.morphologyEx(LoG, cv2.MORPH_ERODE, np.ones((3, 3)))
maxLoG = cv2.morphologyEx(LoG, cv2.MORPH_DILATE, np.ones((3, 3)))    #zero translation approximation

zero_cross = np.logical_and(minLoG < -10, maxLoG > 10)    #true edge condition
zero_cross_im = np.zeros(zero_cross.shape, np.uint8)
zero_cross_im[zero_cross] = 255
cv2.imshow('zero_cross_im', zero_cross_im)

cv2.waitKey(0)
cv2.destroyAllWindows()